In [ ]:
import spacy

complainttext = open("~/Documents/Mymeuron/complaints.txt}, OpenTextMode = "r", encoding = "str")
print(complainttext.read())


FileNotFoundError: [Errno 2] No such file or directory: '~/Documents/Mymeuron/complaints.txt'